### Note to future self: The following shown below has not been done.  The R squared values are 1.0 and this may not be right.  You will be speaking to a Q&A session tonight, 2/27/19 at 6pm to find out more.
### Iterate.  
### Based on the results of your validation test, create a revised model, and then test both old and new models on a new holdout or set of folds.
### Include your model(s) and a brief writeup of the reasoning behind the validation method you chose and the changes you made to submit and review with your mentor.

In [1]:
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import scipy
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
import datetime
import collections
import statsmodels.stats.proportion
import warnings
from sklearn import linear_model

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

%matplotlib inline
sns.set_style('dark')

In [2]:
#New York State 2013 statistics
#importing and transforming the target file from excel spreadsheet to csv
data_xls = pd.read_excel('/Users/manueltanguma/Downloads/crimes.xls', index_col=3)
data_xls.to_csv('crimes.csv', encoding='utf-8')

In [3]:
#the raw target csv file
data = pd.read_csv('crimes.csv', encoding='latin-1')

In [4]:
#making row 3 the column titles
data = data.iloc[3:]

In [5]:
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

In [6]:
#reseting the index
data = data.reset_index(drop=True)

In [7]:
data

3,Murder and nonnegligent manslaughter,City,Population,Violent crime,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,0,Adams Village,1861,0,NaN,0,0,0,12,2,10,0,0
1,0,Addison Town and Village,2577,3,NaN,0,0,3,24,3,20,1,0
2,0,Akron Village,2846,3,NaN,0,0,3,16,1,15,0,0
3,8,Albany,97956,791,NaN,30,227,526,4090,705,3243,142,NaN
4,0,Albion Village,6388,23,NaN,3,4,16,223,53,165,5,NaN
5,0,Alfred Village,4089,5,NaN,0,3,2,46,10,36,0,NaN
6,0,Allegany Village,1781,3,NaN,0,0,3,10,0,10,0,0
7,1,Amherst Town,118296,107,NaN,7,31,68,2118,204,1882,32,3
8,0,Amityville Village,9519,9,NaN,2,4,3,210,16,188,6,1
9,0,Amsterdam,18182,30,NaN,0,12,18,405,99,291,15,0


In [8]:
columnsTitles = ['City', 'Population', 'Murder and\nnonnegligent\nmanslaughter','Violent\ncrime', 'Rape\n(revised\ndefinition)1',
                'Rape\n(legacy\ndefinition)2', 'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
                'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3']

data = data.reindex(columns=columnsTitles)

In [9]:
data.head(5)

3,City,Population,Murder and nonnegligent manslaughter,Violent crime,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0
1,Addison Town and Village,2577,0,3,NaN,0,0,3,24,3,20,1,0
2,Akron Village,2846,0,3,NaN,0,0,3,16,1,15,0,0
3,Albany,97956,8,791,NaN,30,227,526,4090,705,3243,142,NaN
4,Albion Village,6388,0,23,NaN,3,4,16,223,53,165,5,NaN


In [10]:
data.tail(5)

3,City,Population,Murder and nonnegligent manslaughter,Violent crime,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
346,Yonkers,199134,6,1036,NaN,25,390,615,2368,470,1662,236,10
347,Yorktown Town,36643,0,15,NaN,0,2,13,334,45,287,2,NaN
348,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,2 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,3 The FBI does not publish arson data unless i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#getting rid of the last 3 rows
data = data[:-3]

In [12]:
#addressing column names to make them more readable
data.columns = [
    'city', 'population', 'murder', 'violent_crime',
    'rape_revised_definition_1', 'rape_revised_definition_2', 'robbery', 'aggravated_assault',
    'property_crime', 'burglary', 'larceny_theft', 'motorvehicle_theft', 'arson']

In [13]:
#rape_revised_definition_1 seems to be empty of data; therefore will delete the column from the dataframe
data = data.drop(['rape_revised_definition_1'], axis=1)

In [14]:
#re-addressing column names to make them more readable
data.columns = [
    'city', 'population', 'murder', 'violent_crime',
    'rape', 'robbery', 'aggravated_assault',
    'property_crime', 'burglary', 'larceny_theft', 'motorvehicle_theft', 'arson']

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 12 columns):
city                  348 non-null object
population            348 non-null object
murder                348 non-null object
violent_crime         348 non-null object
rape                  348 non-null object
robbery               348 non-null object
aggravated_assault    348 non-null object
property_crime        348 non-null object
burglary              348 non-null object
larceny_theft         348 non-null object
motorvehicle_theft    348 non-null object
arson                 187 non-null object
dtypes: object(12)
memory usage: 32.7+ KB


In [16]:
#when converted the excel file to csv was left with dtypes of "object", had to convert values to integer or float to
#carry out calculations
for i in range(0, len(data.columns)):
    data.iloc[:,i] = pd.to_numeric(data.iloc[:,i], errors='ignore')
    # errors='ignore' lets strings remain as 'non-null objects'


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 12 columns):
city                  348 non-null object
population            348 non-null int64
murder                348 non-null int64
violent_crime         348 non-null int64
rape                  348 non-null int64
robbery               348 non-null int64
aggravated_assault    348 non-null int64
property_crime        348 non-null int64
burglary              348 non-null int64
larceny_theft         348 non-null int64
motorvehicle_theft    348 non-null int64
arson                 187 non-null float64
dtypes: float64(1), int64(10), object(1)
memory usage: 32.7+ KB


In [18]:
data.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0.000
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0.000
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,nan
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,nan


In [19]:
#fill in NaN of arson with average of column
data['arson'].mean()

1.8716577540106951

In [20]:
data['arson'].fillna((data['arson'].mean()), inplace=True)

In [21]:
data.head(10)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0.000
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0.000
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,1.872
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,1.872
5,Alfred Village,4089,0,5,0,3,2,46,10,36,0,1.872
6,Allegany Village,1781,0,3,0,0,3,10,0,10,0,0.000
7,Amherst Town,118296,1,107,7,31,68,2118,204,1882,32,3.000
8,Amityville Village,9519,0,9,2,4,3,210,16,188,6,1.000
9,Amsterdam,18182,0,30,0,12,18,405,99,291,15,0.000


In [22]:
#converting arson from dtype float 64 to integer 64
data['arson'] = data['arson'].apply(np.int64)

In [23]:
data['population_squared'] = data['population']**2

In [24]:
data.head(10)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,3463321
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0,6640929
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0,8099716
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,1,9595377936
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,1,40806544
5,Alfred Village,4089,0,5,0,3,2,46,10,36,0,1,16719921
6,Allegany Village,1781,0,3,0,0,3,10,0,10,0,0,3171961
7,Amherst Town,118296,1,107,7,31,68,2118,204,1882,32,3,13993943616
8,Amityville Village,9519,0,9,2,4,3,210,16,188,6,1,90611361
9,Amsterdam,18182,0,30,0,12,18,405,99,291,15,0,330585124


In [25]:
data.shape

(348, 13)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 13 columns):
city                  348 non-null object
population            348 non-null int64
murder                348 non-null int64
violent_crime         348 non-null int64
rape                  348 non-null int64
robbery               348 non-null int64
aggravated_assault    348 non-null int64
property_crime        348 non-null int64
burglary              348 non-null int64
larceny_theft         348 non-null int64
motorvehicle_theft    348 non-null int64
arson                 348 non-null int64
population_squared    348 non-null int64
dtypes: int64(12), object(1)
memory usage: 35.4+ KB


In [27]:
#checking for any null values
data.isnull().sum()

city                  0
population            0
murder                0
violent_crime         0
rape                  0
robbery               0
aggravated_assault    0
property_crime        0
burglary              0
larceny_theft         0
motorvehicle_theft    0
arson                 0
population_squared    0
dtype: int64

In [28]:
def func(x):
    if x == 0:
        return int('0')
    elif x > 0:
        return int('1')

data['robbery_cat'] = data['robbery'].apply(func)
data['murder_cat'] = data['murder'].apply(func)

In [29]:
data.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,robbery_cat,murder_cat
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,3463321,0,0
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0,6640929,0,0
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0,8099716,0,0
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,1,9595377936,1,1
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,1,40806544,1,0


In [30]:
data['propertycrime'] = data['population']+data['population_squared']+data['murder_cat']+data['robbery_cat']

In [31]:
data.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,robbery_cat,murder_cat,propertycrime
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,3463321,0,0,3465182
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0,6640929,0,0,6643506
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0,8099716,0,0,8102562
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,1,9595377936,1,1,9595475894
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,1,40806544,1,0,40812933


### Statistical tests are useful for making sure a model is a good fit to the test data, and that all the features are useful to the model. However, to make sure a model has good predictive validity for new data, it is necessary to assess the performance of the model on new datasets.  Importing and working with the New York State 2014 crime statistics to assess the performance of the model.

In [32]:
#New York State 2014 statistics
#importing and transforming the target file from excel spreadsheet to csv
data_xls = pd.read_excel('/Users/manueltanguma/Downloads/Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.xls', index_col=3)
data_xls.to_csv('crimes2014.csv', encoding='utf-8')

In [33]:
#the raw target csv file
data2014 = pd.read_csv('crimes2014.csv', encoding='latin-1')

In [34]:
data2014

,Unnamed: 0,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NEW YORK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Offenses Known to Law Enforcement,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"by City, 2014",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Murder and\nnonnegligent\nmanslaughter,City,Population,Violent\ncrime,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,0,Adams Village,1851,0,NaN,0,0,0,11,1,10,0,0
5,0,Addison Town and Village,2568,2,NaN,0,1,1,49,1,47,1,0
6,0,Afton Village4,820,0,0,NaN,0,0,1,0,1,0,0
7,0,Akron Village,2842,1,NaN,0,0,1,17,0,17,0,0
8,8,Albany4,98595,802,54,NaN,237,503,3888,683,3083,122,12
9,0,Albion Village4,5872,26,3,NaN,2,21,204,41,159,4,0


In [35]:
#making row 3 the column titles
data2014 = data2014.iloc[3:]

In [36]:
data2014

,Unnamed: 0,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
3,Murder and\nnonnegligent\nmanslaughter,City,Population,Violent\ncrime,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,0,Adams Village,1851,0,NaN,0,0,0,11,1,10,0,0
5,0,Addison Town and Village,2568,2,NaN,0,1,1,49,1,47,1,0
6,0,Afton Village4,820,0,0,NaN,0,0,1,0,1,0,0
7,0,Akron Village,2842,1,NaN,0,0,1,17,0,17,0,0
8,8,Albany4,98595,802,54,NaN,237,503,3888,683,3083,122,12
9,0,Albion Village4,5872,26,3,NaN,2,21,204,41,159,4,0
10,0,Alexandria Bay Village4,1107,0,0,NaN,0,0,7,2,5,0,0
11,1,Alfred Village4,4032,11,1,NaN,0,9,30,6,24,0,0
12,0,Altamont Village4,1723,1,0,NaN,0,1,2,2,0,0,0


In [37]:
new_header1 = data2014.iloc[0] #grab the first row for the header
data2014 = data2014[1:] #take the data less the header row
data2014.columns = new_header1 #set the header row as the df header

In [38]:
#resetting the index
data2014 = data2014.reset_index(drop=True)

In [39]:
data2014

3,Murder and nonnegligent manslaughter,City,Population,Violent crime,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,0,Adams Village,1851,0,NaN,0,0,0,11,1,10,0,0
1,0,Addison Town and Village,2568,2,NaN,0,1,1,49,1,47,1,0
2,0,Afton Village4,820,0,0,NaN,0,0,1,0,1,0,0
3,0,Akron Village,2842,1,NaN,0,0,1,17,0,17,0,0
4,8,Albany4,98595,802,54,NaN,237,503,3888,683,3083,122,12
5,0,Albion Village4,5872,26,3,NaN,2,21,204,41,159,4,0
6,0,Alexandria Bay Village4,1107,0,0,NaN,0,0,7,2,5,0,0
7,1,Alfred Village4,4032,11,1,NaN,0,9,30,6,24,0,0
8,0,Altamont Village4,1723,1,0,NaN,0,1,2,2,0,0,0
9,1,Amherst Town4,118860,128,16,NaN,43,68,2066,176,1846,44,2


In [40]:
columnsTitles1 = ['City', 'Population', 'Murder and\nnonnegligent\nmanslaughter','Violent\ncrime', 'Rape\n(revised\ndefinition)1',
                'Rape\n(legacy\ndefinition)2', 'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
                'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3']

data2014 = data2014.reindex(columns=columnsTitles1)

In [41]:
#getting rid of the last 3 rows
data2014 = data2014[:-7]

In [42]:
#addressing column names to make them more readable
data2014.columns = [
    'city', 'population', 'murder', 'violent_crime',
    'rape_revised_definition_1', 'rape_revised_definition_2', 'robbery', 'aggravated_assault',
    'property_crime', 'burglary', 'larceny_theft', 'motorvehicle_theft', 'arson']

In [43]:
data2014.tail(5)

,city,population,murder,violent_crime,rape_revised_definition_1,rape_revised_definition_2,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson
364,"Woodbury Town4, 5",10739,0,4,0,NaN,1,3,NaN,5,NaN,0,0
365,Woodstock Town4,5907,0,3,1,NaN,2,0,43,14,29,0,0
366,Yonkers4,200624,3,974,33,NaN,358,580,2009,414,1395,200,15
367,Yorktown Town4,36989,0,13,0,NaN,0,13,209,24,182,3,0
368,Youngstown Village4,1896,0,0,0,NaN,0,0,1,0,1,0,0


In [44]:
data2014.rape_revised_definition_2.values

array(['0', '0', nan, '0', nan, nan, nan, nan, nan, nan, nan, '0', '0',
       nan, '0', '0', '0', '0', nan, nan, nan, nan, nan, nan, nan, nan,
       nan, '0', nan, '0', '0', '0', '0', '0', nan, nan, '0', '0', nan,
       nan, nan, '0', '0', '0', nan, nan, '0', nan, nan, '0', nan, '0',
       nan, nan, '0', nan, '0', nan, nan, nan, nan, nan, nan, nan, nan,
       '0', '0', nan, nan, nan, nan, '0', nan, nan, nan, nan, nan, nan,
       nan, '0', '0', '0', nan, nan, nan, nan, nan, '0', '0', nan, '0',
       nan, '0', nan, '0', nan, '0', '0', '0', nan, nan, nan, nan, '0',
       nan, '0', nan, nan, nan, nan, nan, '0', nan, nan, nan, '0', '0',
       nan, '0', '0', '0', '0', '0', '0', nan, nan, nan, '0', '0', '0',
       nan, nan, '0', nan, nan, '0', nan, nan, nan, nan, nan, nan, '0',
       nan, nan, '0', '0', '0', '0', nan, nan, '0', nan, nan, nan, nan,
       '0', '0', nan, '0', nan, '0', nan, '0', nan, nan, '0', '0', nan,
       '0', nan, '0', nan, nan, nan, '0', nan, nan, nan, nan, na

In [45]:
#rape_revised_definition_2 seems to be empty of data; therefore will delete the column from the dataframe
data2014 = data2014.drop(['rape_revised_definition_2'], axis=1)

In [46]:
data2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 12 columns):
city                         369 non-null object
population                   369 non-null object
murder                       369 non-null object
violent_crime                369 non-null object
rape_revised_definition_1    227 non-null object
robbery                      369 non-null object
aggravated_assault           369 non-null object
property_crime               368 non-null object
burglary                     369 non-null object
larceny_theft                368 non-null object
motorvehicle_theft           369 non-null object
arson                        365 non-null object
dtypes: object(12)
memory usage: 34.7+ KB


In [47]:
data2014.arson.values

array(['0', '0', '0', '0', '12', '0', '0', '0', '0', '2', '0', '1', '3',
       '0', '0', nan, '0', '2', '0', '0', '0', '3', '1', '0', '1', '3',
       '5', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', nan,
       '0', '0', '0', '0', '0', '1', '0', '2', '1', '1', '0', '0', '0',
       '0', '0', '0', '0', '0', '8', '0', '0', '0', '0', '2', '0', '0',
       '0', '0', '7', '9', '0', '0', '0', '0', '1', '2', '0', '0', '0',
       '0', '0', '0', '2', '0', '1', '0', '0', '5', '0', '0', '0', '0',
       '0', '0', '0', '1', '3', '3', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '3', '0', '1', '0',
       '0', '0', '0', '5', '0', '1', '0', '1', '2', '2', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '1', '3', '1', '0', '0',
       '1', '0', '1', '0', '3', '2', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '3', '0', '15', '1', '2', '0', '0',
       '0', '0', '0', '1', '0', '7', '0', '1', '0', '2', '0', 

In [48]:
#re-addressing column names to make them more readable
data2014.columns = [
    'city', 'population', 'murder', 'violent_crime',
    'rape', 'robbery', 'aggravated_assault',
    'property_crime', 'burglary', 'larceny_theft', 'motorvehicle_theft', 'arson']

In [49]:
#when converted the excel file to csv was left with dtypes of "object", had to convert values to integer or float to
#carry out calculations
for i in range(0, len(data2014.columns)):
    data2014.iloc[:,i] = pd.to_numeric(data2014.iloc[:,i], errors='ignore')
    # errors='ignore' lets strings remain as 'non-null objects'


In [50]:
data2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 12 columns):
city                  369 non-null object
population            369 non-null int64
murder                369 non-null int64
violent_crime         369 non-null int64
rape                  227 non-null float64
robbery               369 non-null int64
aggravated_assault    369 non-null int64
property_crime        368 non-null float64
burglary              369 non-null int64
larceny_theft         368 non-null float64
motorvehicle_theft    369 non-null int64
arson                 365 non-null float64
dtypes: float64(4), int64(7), object(1)
memory usage: 34.7+ KB


In [51]:
#fill in NaN of arson with average of column
data2014['arson'].mean()

1.4246575342465753

In [52]:
data2014['arson'].fillna((data2014['arson'].mean()), inplace=True)

In [53]:
data2014['property_crime'].mean()

698.3614130434783

In [54]:
data2014['property_crime'].fillna((data2014['property_crime'].mean()), inplace=True)

In [55]:
data2014['larceny_theft'].mean()

562.7907608695652

In [56]:
data2014['larceny_theft'].fillna((data2014['larceny_theft'].mean()), inplace=True)

In [57]:
#converting dtype from float 64 to integer 64
data2014['property_crime'] = data2014['property_crime'].apply(np.int64)
data2014['larceny_theft'] = data2014['larceny_theft'].apply(np.int64)
data2014['arson'] = data2014['arson'].apply(np.int64)

In [58]:
data2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 12 columns):
city                  369 non-null object
population            369 non-null int64
murder                369 non-null int64
violent_crime         369 non-null int64
rape                  227 non-null float64
robbery               369 non-null int64
aggravated_assault    369 non-null int64
property_crime        369 non-null int64
burglary              369 non-null int64
larceny_theft         369 non-null int64
motorvehicle_theft    369 non-null int64
arson                 369 non-null int64
dtypes: float64(1), int64(10), object(1)
memory usage: 34.7+ KB


In [59]:
data2014['rape'].mean()

15.555066079295154

In [60]:
data2014['rape'] = data2014['rape'].replace('nan', np.nan).fillna(0)

In [61]:
data2014['rape'] = data2014['rape'].apply(np.int64)

In [62]:
data2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 12 columns):
city                  369 non-null object
population            369 non-null int64
murder                369 non-null int64
violent_crime         369 non-null int64
rape                  369 non-null int64
robbery               369 non-null int64
aggravated_assault    369 non-null int64
property_crime        369 non-null int64
burglary              369 non-null int64
larceny_theft         369 non-null int64
motorvehicle_theft    369 non-null int64
arson                 369 non-null int64
dtypes: int64(11), object(1)
memory usage: 34.7+ KB


In [63]:
data2014.shape

(369, 12)

In [64]:
data2014.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson
0,Adams Village,1851,0,0,0,0,0,11,1,10,0,0
1,Addison Town and Village,2568,0,2,0,1,1,49,1,47,1,0
2,Afton Village4,820,0,0,0,0,0,1,0,1,0,0
3,Akron Village,2842,0,1,0,0,1,17,0,17,0,0
4,Albany4,98595,8,802,54,237,503,3888,683,3083,122,12


In [65]:
data.shape

(348, 16)

In [66]:
data.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,robbery_cat,murder_cat,propertycrime
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,3463321,0,0,3465182
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0,6640929,0,0,6643506
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0,8099716,0,0,8102562
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,1,9595377936,1,1,9595475894
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,1,40806544,1,0,40812933


In [67]:
#creating columns as per required in the assignment
data2014['population_squared'] = data2014['population']**2
data2014['propertycrime'] = data2014['population']+data2014['population_squared']+data2014['murder']+data2014['robbery']

In [68]:
#creating columns as per required in the assignment; making robbery and murder categorical
def func(x):
    if x == 0:
        return int('0')
    elif x > 0:
        return int('1')

data2014['robbery_cat'] = data2014['robbery'].apply(func)
data2014['murder_cat'] = data2014['murder'].apply(func)

In [69]:
data2014.shape

(369, 16)

In [70]:
data2014.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,propertycrime,robbery_cat,murder_cat
0,Adams Village,1851,0,0,0,0,0,11,1,10,0,0,3426201,3428052,0,0
1,Addison Town and Village,2568,0,2,0,1,1,49,1,47,1,0,6594624,6597193,1,0
2,Afton Village4,820,0,0,0,0,0,1,0,1,0,0,672400,673220,0,0
3,Akron Village,2842,0,1,0,0,1,17,0,17,0,0,8076964,8079806,0,0
4,Albany4,98595,8,802,54,237,503,3888,683,3083,122,12,9720974025,9721072865,1,1


In [71]:
data2014.tail(25)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,propertycrime,robbery_cat,murder_cat
344,Warsaw Village4,3397,0,5,1,0,4,75,6,69,0,1,11539609,11543006,0,0
345,Washingtonville Village,5788,0,2,0,0,2,59,5,53,1,0,33500944,33506732,0,0
346,Waterford Town and Village,8398,0,1,0,1,0,58,14,44,0,0,70526404,70534803,1,0
347,Waterloo Village4,5185,0,13,2,1,10,154,22,127,5,0,26884225,26889411,1,0
348,Watertown4,28009,0,153,47,17,89,1296,193,1097,6,3,784504081,784532107,1,0
349,Watervliet4,10235,0,20,2,10,8,199,32,152,15,0,104755225,104765470,1,0
350,Watkins Glen Village4,1871,0,2,0,0,2,103,10,93,0,0,3500641,3502512,0,0
351,Waverly Village4,4344,0,4,2,0,2,20,6,13,1,0,18870336,18874680,0,0
352,Webster Town and Village4,44230,1,27,6,7,13,496,67,418,11,1,1956292900,1956337138,1,1
353,Weedsport Village,1796,0,2,0,0,2,21,7,14,0,0,3225616,3227412,0,0


In [72]:
#selecting the data as per the instructions in the assignment; this model includes 'robbery_cat' and 'murder_cat'
select_data2014 = data2014[['population', 'population_squared', 'murder_cat', 'robbery_cat', 'propertycrime']]
#below is the select_data for the original 2013 NY State Crime dataset
select_data = data[['population', 'population_squared', 'murder_cat', 'robbery_cat', 'propertycrime']]

In [73]:
select_data2014.head(5)

,population,population_squared,murder_cat,robbery_cat,propertycrime
0,1851,3426201,0,0,3428052
1,2568,6594624,0,1,6597193
2,820,672400,0,0,673220
3,2842,8076964,0,0,8079806
4,98595,9720974025,1,1,9721072865


In [74]:
select_data.head(5)

,population,population_squared,murder_cat,robbery_cat,propertycrime
0,1861,3463321,0,0,3465182
1,2577,6640929,0,0,6643506
2,2846,8099716,0,0,8102562
3,97956,9595377936,1,1,9595475894
4,6388,40806544,0,1,40812933


In [75]:
select_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 5 columns):
population            348 non-null int64
population_squared    348 non-null int64
murder_cat            348 non-null int64
robbery_cat           348 non-null int64
propertycrime         348 non-null int64
dtypes: int64(5)
memory usage: 13.7 KB


In [76]:
select_data2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 5 columns):
population            369 non-null int64
population_squared    369 non-null int64
murder_cat            369 non-null int64
robbery_cat           369 non-null int64
propertycrime         369 non-null int64
dtypes: int64(5)
memory usage: 14.5 KB


In [77]:
#Troubleshooting why some values in this column are Null
select_data2014['propertycrime'].isnull()

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
339    False
340    False
341    False
342    False
343    False
344    False
345    False
346    False
347    False
348    False
349    False
350    False
351    False
352    False
353    False
354    False
355    False
356    False
357    False
358    False
359    False
360    False
361    False
362    False
363    False
364    False
365    False
366    False
367    False
368    False
Name: propertycrime, Length: 369, dtype: bool

In [78]:
select_data2014.tail(20)

,population,population_squared,murder_cat,robbery_cat,propertycrime
349,10235,104755225,0,1,104765470
350,1871,3500641,0,0,3502512
351,4344,18870336,0,0,18874680
352,44230,1956292900,1,1,1956337138
353,1796,3225616,0,0,3227412
354,4576,20939776,0,0,20944352
355,3147,9903609,0,0,9906756
356,1753,3073009,0,0,3074762
357,44866,2012957956,0,1,2013002839
358,2598,6749604,0,0,6752202


In [79]:
#finding how many rows in propertycrime are Null
select_data2014['propertycrime'].isnull().sum(axis = 0)

0

In [80]:
#seperating the dataset into features and labels
X1 = select_data2014[['population', 'population_squared', 'murder_cat', 'robbery_cat']]
y1 = select_data2014['propertycrime']
X = select_data[['population', 'population_squared', 'murder_cat', 'robbery_cat']]
y = select_data['propertycrime']

In [81]:
X.head(5)

,population,population_squared,murder_cat,robbery_cat
0,1861,3463321,0,0
1,2577,6640929,0,0
2,2846,8099716,0,0
3,97956,9595377936,1,1
4,6388,40806544,0,1


In [82]:
X1.head(5)

,population,population_squared,murder_cat,robbery_cat
0,1851,3426201,0,0
1,2568,6594624,0,1
2,820,672400,0,0
3,2842,8076964,0,0
4,98595,9720974025,1,1


In [83]:
y.head(5)

0       3465182
1       6643506
2       8102562
3    9595475894
4      40812933
Name: propertycrime, dtype: int64

In [84]:
y1.head(5)

0       3428052
1       6597193
2        673220
3       8079806
4    9721072865
Name: propertycrime, dtype: int64

In [85]:
from sklearn import preprocessing

In [86]:
X = preprocessing.scale(X)
X1 = preprocessing.scale(X1)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [87]:
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [88]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.20)
X1_train, X1_test, y1_train, y1_test = cross_validation.train_test_split(X1, y1, test_size=0.20)

In [89]:
#creates linear regression class object for the 2014 data and 2013 data
regr2014 = linear_model.LinearRegression()
regr = linear_model.LinearRegression()

In [90]:
#fitting the 4 independent variables and the 1 dependent variable
#regr.fit(select_data[['population', 'population_squared', 'murder_cat', 'robbery_cat']], select_data.propertycrime)

In [91]:
regr2014.fit(X1_train, y1_train)
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [92]:
accuracy = regr.score(X_test, y_test)
accuracy2014 = regr2014.score(X1_test, y1_test)

In [93]:
print (accuracy)
print (accuracy2014)

1.0
1.0


In [94]:
#Predict the output
y_pred=regr.predict(X_test)
y1_pred=regr2014.predict(X1_test)

In [95]:
print (y_pred)

[  1.59320730e+07   6.65382000e+06   1.04254723e+09   9.07515600e+06
   1.62435000e+06   2.48986030e+09   6.12697570e+07   6.40400070e+07
   2.93601430e+07   3.84356000e+06   5.14450443e+08   8.10256200e+06
   8.60160913e+08   2.67858010e+07   2.53653403e+08   3.63187020e+07
   4.17609661e+08   9.10783920e+07   8.58309610e+07   2.51652720e+07
   7.43680172e+08   4.33622001e+05   1.82655200e+06   2.48253060e+07
   1.38357961e+09   7.60462482e+09   4.08129330e+07   1.14179911e+08
   4.92618000e+06   2.71805830e+07   2.12649307e+08   5.82232772e+08
   2.21323200e+07   5.71616373e+08   1.04970272e+08   1.91144451e+08
   6.87226300e+06   3.23100600e+06   1.40164128e+09   3.67278061e+08
   6.36245520e+07   9.47316064e+08   3.46518200e+06   4.20876570e+07
   2.88315320e+07   7.27864930e+07   1.10523000e+07   8.48265700e+06
   3.33975600e+06   7.46655600e+06   1.06961700e+07   4.88299507e+08
   1.47318907e+08   2.30720911e+08   7.73368292e+08   5.40795000e+06
   1.33945903e+08   1.04550600e+06

In [96]:
print (y1_pred)

[  6.31160635e+03   8.34573531e+07   2.79390685e+06   2.37421878e+08
   7.62035706e+06   9.04305839e+07   2.04346435e+06   6.04053531e+08
   6.96807644e+07   4.62357524e+08   1.63984497e+07   1.61607673e+08
   7.44337481e+07   1.86922206e+09   1.23727877e+07   7.84532122e+08
   1.46712655e+08   1.05565345e+08   2.21323041e+07   3.37601681e+09
   5.32453523e+06   1.73347319e+07   1.15336856e+08   9.50273147e+08
   9.11133959e+06   2.55839470e+06   8.78825748e+06   7.14616706e+07
   5.80014996e+08   1.40044356e+09   9.50796972e+06   1.20461621e+08
   9.65299444e+05   4.33418151e+09   3.23459710e+06   1.05008380e+07
   7.12251517e+07   1.17613365e+06   2.31216563e+07   3.19620480e+07
   2.31754958e+08   1.43451456e+07   6.00702403e+07   1.08405541e+07
   3.47451165e+07   4.36194249e+07   2.84462243e+07   4.48029375e+07
   5.61808536e+08   9.17179270e+06   1.12986743e+09   3.35067352e+07
   1.32975490e+08   1.62610630e+07   6.19644494e+09   4.65192253e+07
   4.81844107e+07   7.44337555e+07

In [97]:
#Print the co-efficient and intercept of the equation
print(regr.coef_)
print(regr.intercept_)
print(regr2014.coef_)
print(regr2014.intercept_)

[  4.49390297e+05   3.77344222e+12   3.47332603e-01   4.89340877e-01]
203554690999.0
[  4.41457435e+05   3.73304209e+12   2.54145894e+00  -8.43830951e+00]
195523883798.0


In [98]:
#Predict the score(% Accuracy)
print('Train score of 2013: ', regr.score(X_train, y_train))
print("Test score of 2013: ", regr.score(X_test, y_test))
print('Train score of 2014: ', regr2014.score(X1_train, y1_train))
print("Test score of 2014: ", regr2014.score(X1_test, y1_test))

Train score of 2013:  1.0
Test score of 2013:  1.0
Train score of 2014:  1.0
Test score of 2014:  1.0


In [99]:
#Calculating the R^2 value
from sklearn import metrics
print ('The R squared value of the 2013 data is: ', metrics.r2_score(y_test, y_pred))
print ('The R squared value of the 2014 data is: ', metrics.r2_score(y1_test, y1_pred))

The R squared value of the 2013 data is:  1.0
The R squared value of the 2014 data is:  1.0
